# MVP: Modelagem de Ameaças com IA (STRIDE)

Este notebook cria um MVP para detectar componentes em diagramas de arquitetura e gerar ameaças e contramedidas baseadas em STRIDE.

## 1) Setup

In [ ]:
!apt-get -qq update && apt-get -qq install -y tesseract-ocr
!pip -q install ultralytics opencv-python pytesseract

In [ ]:
from pathlib import Path
import cv2
import pytesseract
from ultralytics import YOLO

## 2) Estrutura do dataset
Use formato YOLO ou COCO. Exemplo para YOLO:
- data/images/train
- data/images/val
- data/labels/train
- data/labels/val

In [ ]:
DIR_DADOS = Path('/content/data')
print('Diretório de dados:', DIR_DADOS)

## 3) Treinamento do detector (YOLOv8)

In [ ]:
# data.yaml deve definir caminhos de treino/val e nomes das classes
# Exemplo de classes: user, server, database, api, gateway, queue, storage, third_party, auth, frontend, backend, network
modelo = YOLO('yolov8n.pt')
# modelo.train(data='data.yaml', epochs=50, imgsz=640)

## 4) Inferência

In [ ]:
# resultados = modelo.predict(source='amostra.png', conf=0.25)
# deteccoes = []
# for r in resultados:
#     for c in r.boxes.cls.tolist():
#         deteccoes.append(modelo.names[int(c)])
# print(deteccoes)

## 5) Mapeamento STRIDE

In [ ]:
MAPA_STRIDE = {
    'user': ['Spoofing', 'Elevation of Privilege'],
    'auth': ['Spoofing', 'Elevation of Privilege', 'Repudiation'],
    'api': ['Tampering', 'Repudiation', 'Denial of Service'],
    'gateway': ['Tampering', 'Repudiation', 'Denial of Service'],
    'database': ['Information Disclosure', 'Tampering'],
    'storage': ['Information Disclosure', 'Tampering'],
    'queue': ['Tampering', 'Denial of Service'],
    'frontend': ['Spoofing', 'Tampering', 'Information Disclosure'],
    'backend': ['Tampering', 'Repudiation', 'Denial of Service'],
    'network': ['Spoofing', 'Information Disclosure', 'Denial of Service'],
    'server': ['Tampering', 'Denial of Service', 'Elevation of Privilege'],
    'third_party': ['Spoofing', 'Information Disclosure', 'Repudiation']
}

def mapear_stride(componentes):
    saida = {}
    for c in componentes:
        saida[c] = MAPA_STRIDE.get(c, [])
    return saida

## 6) Geração de relatório

In [ ]:
componentes = ['api', 'database', 'user']
mapeamento = mapear_stride(componentes)

linhas = ['# Relatório de Modelagem de Ameaças', '']
for comp, ameacas in mapeamento.items():
    linhas.append(f'## {comp}')
    if ameacas:
        for a in ameacas:
            linhas.append(f'- {a}')
    else:
        linhas.append('- Nenhuma ameaça mapeada')
    linhas.append('')

relatorio = '\n'.join(linhas)
print(relatorio)

with open('relatorio.md', 'w', encoding='utf-8') as f:
    f.write(relatorio)